<a href="https://colab.research.google.com/github/DHANUSHIYA-8/movie-recommendation-system/blob/main/movie_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load movie data (from MovieLens 100k dataset CSVs)
!wget -q http://files.grouplens.org/datasets/movielens/ml-100k/u.data
!wget -q http://files.grouplens.org/datasets/movielens/ml-100k/u.item

# Read ratings data
ratings = pd.read_csv('u.data', sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'])

# Read movie titles
movies = pd.read_csv('u.item', sep='|', header=None, encoding='latin-1',
                     names=['movie_id', 'title', 'release_date', 'video_release_date',
                            'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation',
                            'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
                            'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
                            'Thriller', 'War', 'Western'])

# Create user-item rating matrix
user_movie_matrix = ratings.pivot(index='user_id', columns='movie_id', values='rating').fillna(0)

# Compute user similarity matrix (cosine similarity)
user_similarity = cosine_similarity(user_movie_matrix)

# Function: Recommend movies for a user based on user similarity (collaborative filtering)
def recommend_movies(user_id, user_movie_matrix, user_similarity, movies, top_n=10):
    user_idx = user_id - 1  # user_id starts at 1
    sim_scores = user_similarity[user_idx]
    weighted_ratings = np.dot(sim_scores, user_movie_matrix)
    sum_sim = np.array([np.abs(sim_scores).sum()]*user_movie_matrix.shape[1])
    predicted_ratings = weighted_ratings / sum_sim

    # Exclude movies the user already rated
    user_rated = user_movie_matrix.iloc[user_idx]
    predicted_ratings[user_rated > 0] = 0

    top_movie_indices = np.argsort(predicted_ratings)[::-1][:top_n]
    top_movie_ids = user_movie_matrix.columns[top_movie_indices]

    return movies[movies['movie_id'].isin(top_movie_ids)][['movie_id', 'title']]

# Example: Recommend for user 1
recommendations = recommend_movies(1, user_movie_matrix, user_similarity, movies)
print("Top 10 movie recommendations for User 1:")
print(recommendations)


Top 10 movie recommendations for User 1:
     movie_id                                   title
285       286             English Patient, The (1996)
287       288                           Scream (1996)
293       294                        Liar Liar (1997)
317       318                 Schindler's List (1993)
356       357  One Flew Over the Cuckoo's Nest (1975)
404       405              Mission: Impossible (1996)
422       423       E.T. the Extra-Terrestrial (1982)
482       483                       Casablanca (1942)
495       496            It's a Wonderful Life (1946)
654       655                      Stand by Me (1986)


In [ ]:
print(ratings.head())


   user_id  movie_id  rating  timestamp
0      196       242       3  881250949
1      186       302       3  891717742
2       22       377       1  878887116
3      244        51       2  880606923
4      166       346       1  886397596


In [ ]:
print(movies[['movie_id', 'title']].head())


   movie_id              title
0         1   Toy Story (1995)
1         2   GoldenEye (1995)
2         3  Four Rooms (1995)
3         4  Get Shorty (1995)
4         5     Copycat (1995)


In [ ]:
print(user_movie_matrix.shape)


(943, 1682)


In [ ]:
# Define user_id
user_id = 1

# Generate recommendations
recommendations = recommend_movies(user_id, user_movie_matrix, user_similarity, movies)

# Show recommendations
print(f"Top 10 movie recommendations for User {user_id}:")
print(recommendations)


Top 10 movie recommendations for User 1:
     movie_id                                   title
285       286             English Patient, The (1996)
287       288                           Scream (1996)
293       294                        Liar Liar (1997)
317       318                 Schindler's List (1993)
356       357  One Flew Over the Cuckoo's Nest (1975)
404       405              Mission: Impossible (1996)
422       423       E.T. the Extra-Terrestrial (1982)
482       483                       Casablanca (1942)
495       496            It's a Wonderful Life (1946)
654       655                      Stand by Me (1986)
